In [ ]:
import os
import shutil
import numpy as np
from google.colab import drive
drive.mount('/content/gdrive')
os.chdir('/content/gdrive/MyDrive/HIDE--main')

In [37]:
from utils import gaussjordan
from LinearCode import LinearCode
import hashlib

def generate_random_vector(n, p):
  """
  Génère un vecteur aléatoire de taille n avec p positions à 1.

  Args:
    n: La taille du vecteur.
    p: Le nombre de positions à 1.

  Returns:
    Un vecteur numpy de taille n avec p positions à 1.
  """
  vector = np.zeros(n, dtype=int)
  indices = np.random.choice(n, p, replace=False)
  vector[indices] = 1
  return vector

In [ ]:
class Moderate_PC(LinearCode):
    """
    Moderate Parity Check Codes representation (extends LinearCode)

    ...

    Methods
    -------
    from_params(n, k)
        Init Moderate-PC by length and dimension

    """

    def __init__(self, G, H):
        super().__init__(G, H)

    @classmethod
    def from_params(cls, n, k):

      I_k = np.eye(k, dtype=int)
      A = np.random.randint(0, 2, size=(n-k, k), dtype=int)
      G = np.concatenate((I_k, A.T), axis=1)
      I_n_k = np.eye(n-k, dtype=int)
      H = np.concatenate((A, I_n_k), axis=1)

      assert (G @ H.T % 2 == 0).all(), "G is not correspond to H"

      return cls(G, H)

class QuasiCyclic_Moderate_PC(Moderate_PC):
    """
    Quasi-Cyclic Moderate Parity Check Codes representation (extends Moderate_PC)

    ...

    Methods
    -------
    from_params(n, k, p)
        Init QuasiCyclic-Moderate-PC by length, dimension and sub-block size

    """

    def __init__(self, G, H):
        super().__init__(G, H)

    @classmethod
    def from_params(cls, n, k, p):
        J = n // p
        I_J = np.eye(J, dtype=int)
        # Generate random permutation matrices
        P_list = [np.random.permutation(I_J) for _ in range(k * (n - k))]
        # Reshape the list into a 3D array
        P_array = np.array(P_list).reshape(n - k, k, J, J)

        A = np.block([[P_array[i, j] for j in range(k)] for i in range(n - k)])
        I_k = np.eye(k * J, dtype=int)
        G = np.concatenate((I_k, A.T), axis=1)
        I_n_k = np.eye((n - k) * J, dtype=int)
        H = np.concatenate((A, I_n_k), axis=1)

        assert (G @ H.T % 2 == 0).all(), "G is not correspond to H"

        return cls(G, H)

# EXAMPLE USAGE:

n = 8
k = 4
p = 4  # Sub-block size
errors_num = 2

quasi_cyclic_moderate_pc = QuasiCyclic_Moderate_PC.from_params(n, k, p)
print(quasi_cyclic_moderate_pc.getG())
print(quasi_cyclic_moderate_pc.getG().shape)

print(quasi_cyclic_moderate_pc.getH())
print(quasi_cyclic_moderate_pc.getH().shape)

word = np.random.randint(2, size=quasi_cyclic_moderate_pc.getG().shape[0])
print(word)
print(len(word))

encoded = quasi_cyclic_moderate_pc.encode(word)
print(encoded)
print(len(encoded))

e = generate_random_vector(len(encoded), errors_num)

print(e)
print(len(e))

corrupted = (encoded + e) % 2
print(corrupted)
print(len(corrupted))

decoded = quasi_cyclic_moderate_pc.decode(np.copy(corrupted))
decoded = quasi_cyclic_moderate_pc.get_message(decoded)
print(decoded)
print(len(decoded))

try:
    assert (decoded == word).all()
except AssertionError:
    print("The secret message must be forwarded")

In [40]:
os.chdir('/content/gdrive/MyDrive/HIDE--main/Animal Species')

In [41]:
folder_path = '/content/gdrive/MyDrive/HIDE--main/Animal Species/file.txt'

folder_names = []

with open(folder_path, 'r') as f:
  for line in f:
    folder_names.append(line.strip()) # strip() removes potential newline characters


# Create the empty folders
for folder_name in folder_names:
  if not os.path.exists(folder_name):
      os.makedirs(folder_name)

In [ ]:
count = 0
for folder_name in folder_names:
    if(os.path.isdir(os.path.join(folder_name))):
        count = count + 1

print(count)

In [ ]:
ls

In [44]:
value = 0
dico = {}
for filename in os.listdir('/content/gdrive/MyDrive/HIDE--main/Animal Species'):
        if(os.path.isdir(filename)):
          dico[value]  = filename
          value = value + 1

In [ ]:
 for i in dico.items():
    print(i)

In [ ]:
y = corrupted
print(y)
len(y)

In [ ]:
lis = []
i = 0

while i < len(y):
    print(dico[i])
    count = 0
    for filename in os.listdir('/content/gdrive/MyDrive/HIDE--main/Animal Species/' + dico[i]):
        print(filename)
        count  = count + 1
    lis.append(count)
    print("-- nombre de fichiers  = ", lis[i],"--")
    print("\n")
    i = i + 1

In [48]:
lis

[3, 2, 3, 5, 3, 3, 4, 2, 5, 7, 4, 2, 4, 3, 5, 4]

In [49]:
def hash_directory(path):
    digest = hashlib.sha256()

    for root, dirs, files in os.walk(path):
        for names in files:
            file_path = os.path.join(root, names)

            # Hash the path and add to the digest to account for empty files/directories
            digest.update(hashlib.sha1(file_path[len(path):].encode()).digest())

            # Per @pt12lol - if the goal is uniqueness over repeatability, this is an alternative method using 'hash'
            # digest.update(str(hash(file_path[len(path):])).encode())

            if os.path.isfile(file_path):
                with open(file_path, 'rb') as f_obj:
                    while True:
                        buf = f_obj.read(1024 * 1024)
                        if not buf:
                            break
                        digest.update(buf)

    return digest.hexdigest()

In [ ]:
list_Hash1 = []
for i in range(len(y)):
    list_Hash1.append('')
list_Hash1

In [ ]:
for i in range(len(y)):
    path = '/content/gdrive/MyDrive/HIDE--main/Animal Species/' + dico[i]
    list_Hash1[i] = hash_directory(path)
    print('--hash '+dico[i],'--',list_Hash1[i])

In [ ]:
y

In [53]:
liste = ['Balistidae', 'Belonidae', 'Enoploteuthidae', 'Cyprinidae', 'Nephropidae', 'Axiidae', 'Serranidae', 'Cardiidae', 'Scombridae', 'Sparidae', 'Acanthuridae', 'Limidae', 'Sergestidae', 'Acipenseridae', 'Acropomatidae', 'Holothuriidae', 'Squalidae', 'Octopodidae', 'Pectinidae', 'Serranidae']

In [ ]:
len(liste)

In [55]:
i = 0
while i < len(y):
    number = np.random.randint(0,20)
    file = '/'+liste[number]+'.txt'
    if(y[i] == 1):
        my_path = '/content/gdrive/MyDrive/HIDE--main/Animal Species/' + dico[i]
        os.chdir(my_path)
        if not os.path.exists(my_path + file):
            with open(my_path + file,'w') as fp:
                pass
        else:
            descriptor = open(my_path + file,'a')
            descriptor.write("Species")
            descriptor.close()
    i = i + 1

In [ ]:
lis = []
i = 0

while i < len(y):
    print(dico[i])
    count = 0
    for filename in os.listdir('/content/gdrive/MyDrive/HIDE--main/Animal Species/' + dico[i]):
        print(filename)
        count  = count + 1
    lis.append(count)
    print("-- nombre de fichiers  = ", lis[i],"--")
    print("\n")
    i = i + 1

In [ ]:
list_Hash2 = []
for i in range(len(y)):
    list_Hash2.append('')
list_Hash2

In [ ]:
for i in range(len(y)):
    path = '/content/gdrive/MyDrive/HIDE--main/Animal Species/' + dico[i]
    list_Hash2[i] = hash_directory(path)
    print('--hash '+dico[i],'--',list_Hash2[i])

In [59]:
msg_secret = []
for i in range(len(y)):
    if(list_Hash1[i] != list_Hash2[i]):
        msg_secret.append(1)
    else:
        msg_secret.append(0)

In [ ]:
np.array(msg_secret)

In [ ]:
y

In [ ]:
len(msg_secret)

In [ ]:
decoded = quasi_cyclic_moderate_pc.decode(np.copy(msg_secret))
decoded = quasi_cyclic_moderate_pc.get_message(decoded)
print(decoded)
print(len(decoded))

In [ ]:
word

In [ ]:
try:
    assert (msg_secret == y).all()
    assert (decoded == word).all()
except AssertionError:
    print("The secret message must be reforwarded")
else:
    print("The secret message has been correctly forwarded")

# #Code script for time evaluation

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
os.chdir('/content/gdrive/MyDrive/HIDE--main')
import time
import os
import shutil
import numpy as np
from utils import gaussjordan
from LinearCode import LinearCode
import hashlib

def generate_random_vector(n, p):
  """
  Génère un vecteur aléatoire de taille n avec p positions à 1.

  Args:
    n: La taille du vecteur.
    p: Le nombre de positions à 1.

  Returns:
    Un vecteur numpy de taille n avec p positions à 1.
  """
  vector = np.zeros(n, dtype=int)
  indices = np.random.choice(n, p, replace=False)
  vector[indices] = 1
  return vector

class Moderate_PC(LinearCode):
    """
    Moderate Parity Check Codes representation (extends LinearCode)

    ...

    Methods
    -------
    from_params(n, k)
        Init Moderate-PC by length and dimension

    """

    def __init__(self, G, H):
        super().__init__(G, H)

    @classmethod
    def from_params(cls, n, k):

      I_k = np.eye(k, dtype=int)
      A = np.random.randint(0, 2, size=(n-k, k), dtype=int)
      G = np.concatenate((I_k, A.T), axis=1)
      I_n_k = np.eye(n-k, dtype=int)
      H = np.concatenate((A, I_n_k), axis=1)

      assert (G @ H.T % 2 == 0).all(), "G is not correspond to H"

      return cls(G, H)

class QuasiCyclic_Moderate_PC(Moderate_PC):
    """
    Quasi-Cyclic Moderate Parity Check Codes representation (extends Moderate_PC)

    ...

    Methods
    -------
    from_params(n, k, p)
        Init QuasiCyclic-Moderate-PC by length, dimension and sub-block size

    """

    def __init__(self, G, H):
        super().__init__(G, H)

    @classmethod
    def from_params(cls, n, k, p):
        J = n // p
        I_J = np.eye(J, dtype=int)
        # Generate random permutation matrices
        P_list = [np.random.permutation(I_J) for _ in range(k * (n - k))]
        # Reshape the list into a 3D array
        P_array = np.array(P_list).reshape(n - k, k, J, J)

        A = np.block([[P_array[i, j] for j in range(k)] for i in range(n - k)])
        I_k = np.eye(k * J, dtype=int)
        G = np.concatenate((I_k, A.T), axis=1)
        I_n_k = np.eye((n - k) * J, dtype=int)
        H = np.concatenate((A, I_n_k), axis=1)

        assert (G @ H.T % 2 == 0).all(), "G is not correspond to H"

        return cls(G, H)

def hash_directory(path):
    digest = hashlib.sha256()

    for root, dirs, files in os.walk(path):
        for names in files:
            file_path = os.path.join(root, names)

            # Hash the path and add to the digest to account for empty files/directories
            digest.update(hashlib.sha1(file_path[len(path):].encode()).digest())

            # Per @pt12lol - if the goal is uniqueness over repeatability, this is an alternative method using 'hash'
            # digest.update(str(hash(file_path[len(path):])).encode())

            if os.path.isfile(file_path):
                with open(file_path, 'rb') as f_obj:
                    while True:
                        buf = f_obj.read(1024 * 1024)
                        if not buf:
                            break
                        digest.update(buf)

    return digest.hexdigest()

os.chdir('/content/gdrive/MyDrive/HIDE--main/Animal Species')
folder_path = '/content/gdrive/MyDrive/HIDE--main/Animal Species/file.txt'

folder_names = []

with open(folder_path, 'r') as f:
  for line in f:
    folder_names.append(line.strip()) # strip() removes potential newline characters


# Create the empty folders
for folder_name in folder_names:
  if not os.path.exists(folder_name):
      os.makedirs(folder_name)

start = time.time()

n = 8
k = 4
p = 4  # Sub-block size
errors_num = 2

quasi_cyclic_moderate_pc = QuasiCyclic_Moderate_PC.from_params(n, k, p)
print(quasi_cyclic_moderate_pc.getG())
print(quasi_cyclic_moderate_pc.getG().shape)

print(quasi_cyclic_moderate_pc.getH())
print(quasi_cyclic_moderate_pc.getH().shape)

word = np.random.randint(2, size=quasi_cyclic_moderate_pc.getG().shape[0])
print(word)
print(len(word))

encoded = quasi_cyclic_moderate_pc.encode(word)
print(encoded)
print(len(encoded))

e = generate_random_vector(len(encoded), errors_num)

print(e)
print(len(e))

corrupted = (encoded + e) % 2
print(corrupted)
print(len(corrupted))


value = 0
dico = {}
for filename in os.listdir('/content/gdrive/MyDrive/HIDE--main/Animal Species'):
        if(os.path.isdir(filename)):
          dico[value]  = filename
          value = value + 1

y = corrupted

list_Hash1 = []
for i in range(len(y)):
    list_Hash1.append('')


for i in range(len(y)):
    path = '/content/gdrive/MyDrive/HIDE--main/Animal Species/' + dico[i]
    list_Hash1[i] = hash_directory(path)


liste = ['Balistidae', 'Belonidae', 'Enoploteuthidae', 'Cyprinidae', 'Nephropidae', 'Axiidae', 'Serranidae', 'Cardiidae', 'Scombridae', 'Sparidae', 'Acanthuridae', 'Limidae', 'Sergestidae', 'Acipenseridae', 'Acropomatidae', 'Holothuriidae', 'Squalidae', 'Octopodidae', 'Pectinidae', 'Serranidae']

i = 0
while i < len(y):
    number = np.random.randint(0,20)
    file = '/'+liste[number]+'.txt'
    if(y[i] == 1):
        my_path = '/content/gdrive/MyDrive/HIDE--main/Animal Species/' + dico[i]
        os.chdir(my_path)
        if not os.path.exists(my_path + file):
            with open(my_path + file,'w') as fp:
                pass
        else:
            descriptor = open(my_path + file,'a')
            descriptor.write("Species")
            descriptor.close()
    i = i + 1



list_Hash2 = []
for i in range(len(y)):
    list_Hash2.append('')


for i in range(len(y)):
    path = '/content/gdrive/MyDrive/HIDE--main/Animal Species/' + dico[i]
    list_Hash2[i] = hash_directory(path)


msg_secret = []
for i in range(len(y)):
    if(list_Hash1[i] != list_Hash2[i]):
        msg_secret.append(1)
    else:
        msg_secret.append(0)


decoded = quasi_cyclic_moderate_pc.decode(np.copy(msg_secret))
decoded = quasi_cyclic_moderate_pc.get_message(decoded)

end = time.time()
print('time = ',end - start)

try:
    assert (msg_secret == y).all()
    assert (decoded == word).all()
except AssertionError:
    print("The secret message must be reforwarded")
else:
    print("The secret message has been correctly forwarded")